In [5]:
import os
import numpy as np
from PIL import Image
from crop_img import crop_img

In [ ]:
bands = ['B12', 'B11', 'B8A', 'B8', 'B4', 'B3']
IMG_SIZE = (2700, 2700)
PATCH_SIZE = (300,300)

In [ ]:
# Create directories
for f1 in ['all_data']: #, 'vegetation', 'burned', 'other', 'unknown', 'mixed']:
    if not os.path.exists(f"./classes/{f1}/mask/"):
        os.makedirs(f"./classes/{f1}/mask/")
    if not os.path.exists(f"./classes/{f1}/normalized_mask/"):
        os.makedirs(f"./classes/{f1}/normalized_mask/")
    for b in bands:
        if not os.path.exists(f"./classes/{f1}/{b}/"):
            os.makedirs(f"./classes/{f1}/{b}/")

In [ ]:
# Divide train mask and band images into 81 images and save them 
for file in os.listdir('./train_masks/'):
    fs  = file.split('_')
    tile_name = '_'.join(fs[:4])
    file_no = fs[-1][:-4]
    img = Image.open('./train_masks/'+file).resize(IMG_SIZE)
    crop_img(img, 9, f'./classes/all_data/mask/{tile_name}_{file_no}_')
    for b in bands:
        file_path = f'../cropped/bands/{tile_name}_{b}_{file_no}.jpg'
        img = Image.open(file_path).resize(IMG_SIZE)
        crop_img(img, 9, f'./classes/all_data/{b}/{tile_name}_{file_no}_')

In [ ]:
# Threshold area for mixed
threshold = 0.95

In [8]:
# Identify the class for each mask and also saves normalized mask
other = []
burned = []
vegetation = []
unknown = []
mixed = []
for file in os.listdir('./classes/all_data/mask/'):
    img = Image.open('./classes/all_data/mask/'+file)
    # img.show()
    mask = np.zeros(PATCH_SIZE)
    data = np.array(img)

    red, green, blue = data.T
    mask[(red.T > 245) & (green.T < 10) & (blue.T < 10)] = 1       # red
    mask[(red.T < 10) & (green.T > 245) & (blue.T < 10)] = 2       # green
    mask[(red.T > 245) & (green.T > 245) & (blue.T > 245)] = 3     # white

    outimg = Image.fromarray(mask)
    # print(np.mean(mask == 1))
    if np.mean(mask == 0) > threshold:
        other.append(file)
    elif np.mean(mask == 1) > threshold:
        burned.append(file)
    elif np.mean(mask == 2) > threshold:
        vegetation.append(file)
    elif np.mean(mask == 3) > threshold:
        unknown.append(file)
    else:
        mixed.append(file)
    outimg = Image.fromarray(mask).convert('L')
    out_name = file.replace('jpg', 'png')
    outimg.save(f'./classes/all_data/normalized_mask/{out_name}')

In [18]:
# Print number of files for each class
print("other",len(other))
print("burned",len(burned))
print("vegetation",len(vegetation))
print("unknown",len(unknown))
print("mixed",len(mixed))
print('total', len(other+burned+vegetation+unknown+mixed))

other 555
burned 438
vegetation 2524
unknown 469
mixed 3952
total 7938


In [19]:
# Saves file names to corresponding classes
with open('./classes/other.txt', 'w') as f:
    for a in other:
        f.write(a[:-4]+'\n')
with open('./classes/burned.txt', 'w') as f:
    for a in burned:
        f.write(a[:-4]+'\n')
with open('./classes/vegetation.txt', 'w') as f:
    for a in vegetation:
        f.write(a[:-4]+'\n')
with open('./classes/unknown.txt', 'w') as f:
    for a in unknown:
        f.write(a[:-4]+'\n')
with open('./classes/mixed.txt', 'w') as f:
    for a in mixed:
        f.write(a[:-4]+'\n')

In [22]:
# Saves file names corresponding labels to csv
with open('./classes/labels.csv', 'w')as csv:
    csv.write('file,label\n')
    for i, l in enumerate([other, burned, vegetation, unknown, mixed]):
        for file in l:
            csv.write(f'{file},{i}\n')
            # shutil.copyfile(f'./out/mask/{file}', f'./classes/other/mask/{file}')
            # for b in bands:
            #     shutil.copyfile(f'./out/{b}/{file}', f'./classes/other/{b}/{file}')